In [23]:
PROJECT="vertex-ai-by-example"
COMPONENT_NAME="bash-io"
BUCKET_NAME="gs://$PROJECT"

In [25]:
!gcloud config set project $PROJECT
!gcloud services enable notebooks.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable aiplatform.googleapis.com
!echo "Note: it can take a while for service accounts to get created with correct permissions"


Updated property [core/project].
Operation "operations/acf.p2-662478959901-9f44e5a9-2d34-4266-bbd3-42574aaae7f6" finished successfully.
Note: it can take a while for service accounts to get created with correct permissions


In [17]:
!gsutil mb $BUCKET_NAME

Creating gs://vertex-ai-by-example/...


In [13]:
!gcloud builds submit --tag gcr.io/$PROJECT/$COMPONENT_NAME .

Creating temporary tarball archive of 7 file(s) totalling 53.0 KiB before compression.
Uploading tarball of [.] to [gs://vertex-ai-by-example_cloudbuild/source/1636929031.237872-08290d3bee034e8f99d8598a0cd0a047.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/vertex-ai-by-example/locations/global/builds/96429e18-d13d-41b9-9a2c-426286377c26].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/96429e18-d13d-41b9-9a2c-426286377c26?project=662478959901].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "96429e18-d13d-41b9-9a2c-426286377c26"

FETCHSOURCE
Fetching storage object: gs://vertex-ai-by-example_cloudbuild/source/1636929031.237872-08290d3bee034e8f99d8598a0cd0a047.tgz#1636929031612809
Copying gs://vertex-ai-by-example_cloudbuild/source/1636929031.237872-08290d3bee034e8f99d8598a0cd0a047.tgz#1636929031612809...
/ [1 files][  7.0 KiB/  7.0 KiB]                                                
Operation com

In [15]:
!pip install kfp --upgrade

     |████████████████████████████████| 296 kB 3.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 127 kB 5.4 MB/s            
     |████████████████████████████████| 105 kB 7.3 MB/s            
     |████████████████████████████████| 1.6 MB 4.5 MB/s            
     |████████████████████████████████| 54 kB 3.0 MB/s            
     |████████████████████████████████| 52 kB 1.8 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 87 kB 4.5 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 10.1 MB 3.7 MB/s            
     |████████████████████████████████| 52 kB 1.9 MB/s            
  Using cached PyYAML-5.4.1-cp37-cp37

In [27]:
from kfp.v2 import dsl, components, compiler
from kfp import components

bash_io_step = components.load_component_from_file("./component.yaml")

@dsl.pipeline(name="bash-io-pipeline", pipeline_root="$BUCKET_NAME/$IMAGE_NAME")
def bash_io_pipeline(project_id: str):
    bash_io_step(input_1="input1value", parameter_1=4)

jobspec_filename = f"{COMPONENT_NAME}-jobspec.json"

compiler.Compiler().compile(
    pipeline_func=bash_io_pipeline, package_path=jobspec_filename
)
print(jobspec_filename)

bash-io-jobspec.json


In [28]:
import kfp

client = kfp.Client() # if this errors then pipelines hasn't been enabled

Failed to load kube config.


MaxRetryError: HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: /apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f837af33f90>: Failed to establish a new connection: [Errno 111] Connection refused'))